# 옮긴이

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
keras.Model.build?

# 1 케라스 모델

## 1.1 `keras.model`
- '`Model`은 층을 학습 및 추론 기능이 있는 개체로 그룹화 한다.

**서명:**

```python
keras.Model(*args, **kwargs)
```

- Args:
    - `inputs`: 모델의 입력: `keras.Input` 객체 또는 `keras.Input` 객체의 목록.
    - `outputs`: 모델의 출력. 아래 함수형 API 예제 참조하라.
    - `name`: 문자열, 모델의 이름.

## 1.2 `Model`을 인스턴스화 하는 두 가지 방법

### 1.2.1 함수형 API

`Input`에서 시작하는 "함수형 API"를 사용하여, 층 호출을 연결하여 모델의 정방향 경로를 지정하고, 최종적으로 입력 및 출력에서 모델을 생성한다:

```python
import tensorflow as tf

inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
```

참고: 오직 입력 텐서의 사전, 목록 및 튜플만 지원된다. 내포된 입력은 지원되지 않는다 (가령, 목록의 목록 또는 사전의 사전).


중간 텐서를 사용하여 새로운 함수형 API 모델이 또한 생성될 수 있다. 이것은 모델의 하위-컴포넌트를 빠르게 추출 가능하게 한다.

예제:

```python
inputs = keras.Input(shape=(None, None, 3))
processed = keras.layers.RandomCrop(width=32, height=32)(inputs)
conv = keras.layers.Conv2D(filters=2, kernel_size=3)(processed)
pooling = keras.layers.GlobalAveragePooling2D()(conv)
feature = keras.layers.Dense(10)(pooling)

full_model = keras.Model(inputs, feature)
backbone = keras.Model(processed, conv)
activations = keras.Model(conv, feature)
```

`backbone` 및 `activations` 모델은 `keras.Input` 객체로 생성되지 않고, `keras.Input` 객체에서 생성된 텐서로 생성됨에 유의하라.
내부적으로, 층과 가중치는 이러한 모델에서 공유되므로, 사용자는 'full_model'을 훈련하고, 'backbone' 또는 'activations'를 사용하여 특성 추출을 수행할 수 있다.

모델의 입력과 출력도 텐서의 중첩 구조가 될 수 있으며, 생성된 모델은 기존의 모든 API를 지원하는 표준 함수형 API 모델이다.

### 1.2.2 하위 클래스화

`Model` 클래스를 하위 클래스화하여: 이 경우에, `__init__()`에서 층을 정의하고 `call()`에서 모델의 정방향 경로를 구현해야 한다.

```python
import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)

model = MyModel()
```

`Model`을 하위 클래스화 하면, 선택적으로 `call()`에 `training` 인수(참/거짓)를 가질 수 있으며, 이를 사용하여 훈련 및 추론에서 다른 동작을 지정할 수 있다:

```python
import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
    self.dropout = tf.keras.layers.Dropout(0.5)

  def call(self, inputs, training=False):
    x = self.dense1(inputs)
    if training:
      x = self.dropout(x, training=training)
    return self.dense2(x)

model = MyModel()
```

일단 모델이 생성되면, `model.compile()`을 사용하여 손실 및 측정항목으로 모델을 구성하거나, `model.fit()`을 사용하여 모델을 훈련하거나, `model.predict()`를 사용하여 모델로 예측을 수행할 수 있다.

## 1.2 `keras.Model.build`
- 전달받은 입력 형태에 기반하여 모델을 구성

이것은 인스턴스화 시간에 입력이 어떻게 생겼는지 알지 못하는 하위 클래스화 모델에 사용된다.

이 메서드는 독립 실행형 방식으로 `model.build()`를 호출하고자 할 때 존재한다 (실제 데이터에서 모델을 호출하여 빌드하는 대신).
이것은 프레임워크에 의해 호출되지 않을 것이다 (따라서 관련 없는 워크플로에서 예기치 않은 오류가 발생하지 않는다).

**서명:*

```python
keras.Model.build(self, input_shape)
```

- 인자:
 - `input_shape`: 단일 튜플, `TensorShape` 인스턴스, 또는 (튜플, 정수 또는 `TensorShape` 인스턴스 형태의) 목록/사전.

- 발생:
  - `ValueError`:
    1. 사용자가 제공한 데이터가 유효하지 않은 경우 (튜플, 목록, `TensorShape` 또는 사전이 아닌 형태).
    2. 모델이 입력 형태에 독립적인 호출 인수를 요구하는 경우(호출 서명의 위치 또는 키워드 인자).
    3. 모든 층이 제대로 구축되지 않은 경우.
    4. 부동소수점 유형 입력이 층 내에서 지원되지 않는 경우.

  이러한 각각의 경우에, 사용자는 실제 텐서 데이터에서 모델을 호출하여 모델을 구성해야 한다.

## 1.3 `keras.Model.train_step`
- 한 단계에 대한 논리

이 메서드는 사용자 지정 학습 논리를 지원하도록 재정의할 수 있다.
이 메소드를 재작성 하느 법에 대한 구체적인 에제는 [Customizing what happens in fit](
https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit) 참조하라.
이 메소드는 `Model.make_train_function`에 의해 호출된다.
 
이 방법은 훈련의 한 단계에 대한 수학적 논리를 포함해야 한다.
여기에는 일반적으로 저방향 경로, 손실 계산, 역전파 및 측정항목 업데이트가 포함된다.

이 로직이 실행되는 *방법*에 대한 구성 세부 정보(예: `tf.function` 및 `tf.distribute.Strategy` 설정)는 `Model.make_train_function`에 남겨두어야 하며 재정의할 수도 있다.

**서명:**

```python
keras.Model.train_step(self, data)
```

- 인자:
  - `data`: `Tensor`의 내포된 구조.

- 반환:
  - `tf.keras.callbacks.CallbackList.on_train_batch_end`에 전달될 값을 닫고 있는 `dict`.
  일반적으로, `Model`의 측정항목의 값은 반환된다. 예: `{'loss': 0.2, 'accuracy': 0.7}`.

In [17]:
keras.Model.make_train_function?

## 1.3  `keras.Model.make_train_function`
- 학습의 한 단계를 실행하는 함수를 만든다.

이 메서드는 사용자 지정 훈련 논리를 지원하도록 재정의할 수 있습니다. 
이 메서드는 `Model.fit` 및 `Model.train_on_batch`에 의해 호출된다.

일반적으로, 이 메서드는 `tf.function` 및 `tf.distribute.Strategy` 설정을 직접 제어하고 실제 훈련 논리를 `Model.train_step`에 위임한다.

이 함수는 `Model.fit` 또는 `Model.train_on_batch`가 처음 호출될 때 캐시된다. 캐시는 `Model.compile`이 호출될 때 언제든지 지워진다. `force=True`로 캐시를 건너뛰고 함수를 다시 생성할 수 있다.

**서명:**

```python
keras.Model.make_train_function(self, force=False)
```

- 인자:
  - `force`: Whether to regenerate the train function and skip the cached
    function if available.

- 반환:
  - 함수. 이 메서드로 생성된 함수는 `tf.data.Iterator`를 수락하고, `tf.keras.Callbacks.on_train_batch_end`에 전달될 (`{'loss': 0.2, '정확도': 0.7}`와 같은) 값을 담고 있는 `dict`을 반환한다.

## 1.4 `keras.Model.metrics`  프로퍼티
- `compile()`, `add_metric()` API를 사용하여 추가된 모델의 측정항목을 반환

참고: `compile()`에 전달된 측정항목은 `keras.Model`이 실제 데이터에서 훈련/평가된 후에만 사용할 수 있다.

**소스:**

```python
@property
def metrics(self):
    metrics = []
    if self._is_compiled:
        # TODO(omalleyt): `LossesContainer` 및 `MetricsContainer` 개체를 추적하여 속성 이름이 부하를 받지 않도록 한다.
        if self.compiled_loss is not None:
            metrics += self.compiled_loss.metrics
        if self.compiled_metrics is not None:
            metrics += self.compiled_metrics.metrics

    for l in self._flatten_layers():
        metrics.extend(l._metrics)
    return metrics
```

**예제:**

In [18]:
inputs = tf.keras.layers.Input(shape=(3,))
outputs = tf.keras.layers.Dense(2)(inputs)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])
[m.name for m in model.metrics]

[]

In [19]:
x = np.random.random((2, 3))
y = np.random.randint(0, 2, (2, 2))
model.fit(x, y)
[m.name for m in model.metrics]

1/1 [==============================] - 0s 296ms/step - loss: 0.6696 - mae: 0.5577


2022-12-12 20:38:40.156787: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


['loss', 'mae']

In [14]:
inputs = tf.keras.layers.Input(shape=(3,))
d = tf.keras.layers.Dense(2, name='out')
output_1 = d(inputs)
output_2 = d(inputs)
model = tf.keras.models.Model(
   inputs=inputs, outputs=[output_1, output_2])
model.add_metric(
   tf.reduce_sum(output_2), name='mean', aggregation='mean')
model.compile(optimizer="Adam", loss="mse", metrics=["mae", "acc"])
model.fit(x, (y, y))
[m.name for m in model.metrics]

1/1 [==============================] - 0s 379ms/step - loss: 1.0959 - out_loss: 0.5479 - out_1_loss: 0.5479 - out_mae: 0.6199 - out_acc: 1.0000 - out_1_mae: 0.6199 - out_1_acc: 1.0000 - mean: 0.1888


2022-12-12 19:34:28.829857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


['loss',
 'out_loss',
 'out_1_loss',
 'out_mae',
 'out_acc',
 'out_1_mae',
 'out_1_acc',
 'mean']

# 2 케라스 측정 항목

## 2.1 `keras.metrics.Metric`
- 측정 항목의 논리 및 상태를 캡슐화한다.

**서명:**

```python
keras.metrics.Metric(*args, **kwargs)
```

- 인자:
  - `name`: (선택적) 측정항목 인스턴스의 문자열 이름.
  - `dtype`: (선택적) 측정항목 결과의 데이터 유형.
  - `**kwarg`: 추가적인 층 키워드 인자.

## 2.2 독립 실행형 사용:

```python
m = SomeMetric(...)
for input in ...:
  m.update_state(input)
print('Final result: ', m.result().numpy())
```

## 2.3 `compile()` API 와 같이 사용

In [11]:
import numpy as np

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.fit(dataset, epochs=10)

Epoch 1/10


2022-12-12 00:40:00.596770: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-12 00:40:00.598619: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 1s 18ms/step - loss: 253.0998 - categorical_accuracy: 0.0910
Epoch 2/10
32/32 [==============================] - 0s 11ms/step - loss: 1085.3861 - categorical_accuracy: 0.0880
Epoch 3/10
32/32 [==============================] - 0s 10ms/step - loss: 2189.2488 - categorical_accuracy: 0.1130
Epoch 4/10
32/32 [==============================] - 0s 10ms/step - loss: 3733.5840 - categorical_accuracy: 0.0840
Epoch 5/10
32/32 [==============================] - 0s 10ms/step - loss: 5462.9048 - categorical_accuracy: 0.1000
Epoch 6/10
32/32 [==============================] - 0s 10ms/step - loss: 7506.7935 - categorical_accuracy: 0.0830
Epoch 7/10
32/32 [==============================] - 0s 11ms/step - loss: 9725.6650 - categorical_accuracy: 0.1000
Epoch 8/10
32/32 [==============================] - 0s 10ms/step - loss: 12616.7715 - categorical_accuracy: 0.0970
Epoch 9/10
32/32 [==============================] - 0s 10ms/step - loss: 16008.2773 - categorical_a

## 2.3 하위 클래스화를 구현할 때

하위 클래스화를 구현하기 위하여:

* `__init__()`: 모든 상태 변수는 다음과 같이 `self.add_weight()`를 호출하여 이 메서드에서 생성되어야 한다: `self.var = self.add_weight(...)`
* `update_state()`: 다음과 같은 상태 변수에 대한 모든 업데이트가 있다: `self.var.assign_add(...)`
* `result()`: 상태 변수에서 측정항목에 대한 스칼라 값 또는 스칼라 값의 사전을 계산하고 반환한다.

하위 클래스화 구현 예제:

In [22]:
class BinaryTruePositives(tf.keras.metrics.Metric):

    def __init__(self, name='binary_true_positives', **kwargs):
        super(BinaryTruePositives, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.bool)
        y_pred = tf.cast(y_pred, tf.bool)

        values = tf.logical_and(tf.equal(y_true, True), tf.equal(y_pred, True))
        values = tf.cast(values, self.dtype)
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, self.dtype)
            sample_weight = tf.broadcast_to(sample_weight, values.shape)
        values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

# 3 콜백

## 3.1 `keras.callbacks.Callback`
- 새 콜백을 구성하는 데 사용되는 추상 기본 클래스.

모델 훈련 및 추론 수명 주기의 다양한 단계에 연결하기 위해 `fit`, `evaluate`, 및 `predict`과 같은 케라스 메서드에 콜백을 전달할 수 있다.

사용자 콜백을 생성하기 위해, `keras.callbacks.Callback` 하위 클래스화하고 관심있는 단계와 관련된 메소드를 재작성한다. 더 상세한 정보는 https://www.tensorflow.org/guide/keras/custom_callback 참조하라.

**예제:**

In [3]:
training_finished = False

class MyCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
        
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(1,))])

model.compile(loss='mean_squared_error')

model.fit(tf.constant([[1.0]]), tf.constant([[1.0]]),
          callbacks=[MyCallback()])

assert training_finished == True

Metal device set to: Apple M1 Pro


2022-12-12 10:21:31.929216: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-12 10:21:31.929703: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-12-12 10:21:32.065491: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 248ms/step - loss: 2.4394


2022-12-12 10:21:32.218189: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


사용자 훈련 루프에서 `Callback` 객체를 사용하기를 원하면:

1. 함께 호출될 수 있도록 모든 콜백을 단일 `callbacks.CallbackList`로 포장하여 함께 호출될 수 있도록 한다.
2. 루프의 적절한 위치에서 수동으로 모든 `on_*` 메소드를 호출할 필요가 있다. 이것과 같다:

Example:
```
   callbacks =  tf.keras.callbacks.CallbackList([...])
   callbacks.append(...)
   callbacks.on_train_begin(...)
   for epoch in range(EPOCHS):
     callbacks.on_epoch_begin(epoch)
     for i, data in dataset.enumerate():
       callbacks.on_train_batch_begin(i)
       batch_logs = model.train_step(data)
       callbacks.on_train_batch_end(i, batch_logs)
     epoch_logs = ...
     callbacks.on_epoch_end(epoch, epoch_logs)
   final_logs=...
   callbacks.on_train_end(final_logs)
```

- 속성:
    - params: 사전. 훈련 매개변수 (eg. verbosity, batch size, number of epochs...).
    - model: `keras.models.Model`의 인스턴스. 학습 중인 모델의 참조.

콜백 메서드가 인수로 취하는 `logs` 사전에는 현재 배치 또는 에포크와 관련된 양에 대한 키가 포함된다 (메소드-특정 docstrings 참조).

## 3.2 `EarlyStopping` - 조기 종료
- 모니터링되는 측정항목이 개선을 중지하면 교육을 중지

훈련의 목표는 손실을 최소화하는 것으로 가정한다. 이를 통해, 모니터링할 측정항목은 `'loss'`이고 모드는 `'min'`이 될 것이다.
`model.fit()` 훈련 루프는 적용 가능한 경우 `min_delta` 및 `patience`를 고려하여 손실이 더 이상 감소하지 않는지 여부를 모든 에포크의 끝에서 확인한다. 일단, 더 이상 감소하지 않음이 발견되면, `model.stop_training`은 참으로 표식되고 훈련은 종료된다.

모니터링할 양은 `logs` dict에서 사용할 수 있어야 한다. 그렇게 하려면 `model.compile()`에서 손실 또는 측정항목을 전달하라.

**서명:**

```python
keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
)
```

- 인자:
  - `monitor`: 모니터링할 양.
  - `min_delta`: 개선으로 자격을 갖추기 위한 모니터링 양의 최소 변경. 가령, min_delta 미만의 절대(값) 변화는 개선되지 않은 것으로 간주.
  - `patience`: 학습이 중단된 후 개선되지 않은 에포크 개수.
  - `verbose`: 세부 정보 표시 모드, 0 또는 1. 모드 0은 메시지를 출력하지 않고, 모드 1은 콜백이 작업을 수행할 때 메시지를 표시.
  - `mode`: `{"auto", "min", "max"}` 중 하나. 
     `min` 모드에서, 모니터링되는 양의 감소가 멈추면 훈련 중지; `"max"` 모드에서는 모니터링된 양의 증가가 중지하면 훈련 중지; `"auto"` 모드에서, 모니터링되는 양의 이름에서 방향이 자동으로 유추
  - `baseline`: 모니터링되는 양의 기준선 값. 모델이 기준선보다 개선되지 않으면 학습이 중지.
  - `restore_best_weights`: 모니터링된 양의 최상의 값으로 에포크에서 모델 가중치를 복원할지 여부이다. 거짓인 경우 훈련 마지막 단계에서 얻은 모델 가중치를 사용. `baseline`과 관련된 성능에 관계없이 에포크가 복원. `baseline`에서 개선되는 에포크가 없으면, `patience` 에포크 동안 훈련이 실행되고 해당 세트의 최상의 에포크에서 가중치를 복원.

**예제:**

In [5]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
# 세 번의 연속적인 에포크에 대한 손실 개선이 없으면 이 콜백은 훈련을 중단할 것이다.
model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])
model.compile(tf.keras.optimizers.SGD(), loss='mse')
history = model.fit(np.arange(100).reshape(5, 20), np.zeros(5),
                    epochs=10, batch_size=1, callbacks=[callback],
                    verbose=0)
len(history.history['loss'])  # Only 4 epochs are run.

2022-12-12 11:38:52.017690: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-12 11:38:52.148835: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4

## 3.3 `ModelCheckpoint` - 모델 및 모델 가중치 저장
- 케라스 모델 또는 모델 가중치를 특정 빈도로 저장하는 콜백.

`ModelCheckpoint` 콜백은 일정 간격으로 모델 또는 가중치(체크포인트 파일에)를 저장하기 위해 `model.fit()`을 사용하는 훈련과 함께 사용되므로 나중에 모델 또는 가중치를 적재하여 저장된 상태에서 훈련을 계속할 수 있다.

이 콜백이 제공하는 몇 가지 옵션은 다음과 같다:

- 지금까지 "최고의 성능"을 달성한 모델만 유지할 것인지, 아니면 성능에 관계없이 매 에포크가 끝날 때마다 모델을 저장할 것인지 여부
- '최고'의 정의; 모니터링할 양과 최대화 또는 최소화해야 하는지 여부.
- 저장해야 하는 빈도입니다. 현재 콜백은 모든 에포크가 끝날 때 또는 고정된 수의 훈련 배치 후 저장을 지원.
- 가중치만 저장할지, 전체 모델을 저장할지 여부.

참고: `WARNING:tensorflow:Can save best model only with <name> available, skipping`을 얻으면 이것을 올바르게 하는 방법에 대한 자세한 내용은 `monitor` 인수에 대한 설명을 참조하라.

**서명:**

```python
keras.callbacks.ModelCheckpoint(
    filepath,
    monitor: str = 'val_loss',
    verbose: int = 0,
    save_best_only: bool = False,
    save_weights_only: bool = False,
    mode: str = 'auto',
    save_freq='epoch',
    options=None,
    initial_value_threshold=None,
    **kwargs,
)
```

- 인자:
    - `filepath`: 문자열 또는 PathLike, 모델 파일을 저장할 경로. 가령, `filepath = os.path.join(working_dir, 'ckpt', file_name)`. `filepath`에는 epoch의 값과 로그의 키(`on_epoch_end`에 전달됨)로 채워지는 명명된 서식 지정 옵션이 포함될 수 있다. 예를 들어: `filepath`가 `weights.{epoch:02d}-{val_loss:.2f}.hdf5`인 경우, 모델 체크포인트는 epoch 번호와 파일 이름의 유효성 검사 손실과 함께 저장된다. 파일 경로의 디렉터리는 충돌을 피하기 위해 다른 콜백에서 재사용하면 안된다.

    - `monitor`: 모니터링할 측정항목 이름입니다. 일반적으로 측정항목은 `Model.compile` 메서드에 의해 설정됩니다. 참고:
      * 유효성 검사 측정항목을 모니터링하려면 이름 앞에 "`val_`"을 붙인다.
      * "`loss`" 또는 "`val_loss`"를 사용하여 모델의 총 손실을 모니터링한다.
      * "`accuracy`"와 같이 측정항목을 문자열로 지정하는 경우 `"val_"` 접두사가 있거나 없는 동일한 문자열을 전달한다.
      * `metrics.Metric` 개체를 전달하는 경우, `monitor`를 `metric.name`으로 설정해야 한다.
      * 측정항목의 이름이 확실하지 않은 경우 `history = model.fit()`에서 반환된 `history.history` 사전의 내용을 확인할 수 있다.
      * 다중-출력 모델은 측정항목 이름에 추가 접두사를 설정한다.

    - `verbose`: 세부 정보 표시 모드, 0 또는 1. 모드 0은 자동이며 모드 1은 콜백이 작업을 수행할 때 메시지를 표시한다.

    - `save_best_only`: `save_best_only=True`인 경우 모델이 "최상"으로 간주되는 경우에만 저장하고 모니터링된 양에 따라 최신 베스트 모델을 덮어쓰지 않는다. `filepath`에 `{epoch}`와 같은 형식 지정 옵션이 포함되어 있지 않으면 `filepath`는 각각의 더 나은 새 모델로 덮어쓰게 된다.

    - `mode`: {`auto`, `min`, `max`} 중 하나. `save_best_only=True`인 경우 현재 저장 파일을 덮어쓸지 여부는 모니터링되는 양의 최대화 또는 최소화에 따라 결정된다. `val_acc`의 경우 `max`여야 하고 `val_loss`의 경우 `min`이어야 한다. `auto` 모드에서 모니터링되는 양이 `acc`이거나 'fmeasure'로 시작하면 모드가 `max`로 설정되고 나머지 양은 `min`으로 설정된다.

    - `save_weights_only`: 참이면 모델의 가중치만 저장되고(`model.save_weights(filepath)`) 그렇지 않으면 전체 모델이 저장된다(`model.save(filepath)`).

    - `save_freq`: '`epoch`' 또는 정수. '`epoch`'를 사용할 때 콜백은 각 epoch 이후에 모델을 저장한다. 정수를 사용할 때 콜백은 이 많은 배치가 끝날 때 모델을 저장한다. `Model`이 `steps_per_execution=N`으로 컴파일되면 N번째 배치마다 저장 기준이 확인된다. `Model`이 `steps_per_execution=N`으로 컴파일된 경우 저장 기준은 매 N번째 배치마다 확인된다. 메트릭이 매 에포크마다 재설정되기 때문에 1 배치만큼 적게 반영된다. 기본값은 '`epoch`'이다.

    - `options`: `save_weights_only`가 참인 경우 선택적 `tf.train.CheckpointOptions` 개체 또는 `save_weights_only`가 거짓인 경우 선택적 `tf.saved_model.SaveOptions` 개체이다.

    - `initial_value_threshold`: 모니터링할 측정항목의 부동소수점 초기 "최고" 값이다. `save_best_value=True`인 경우에만 적용된다. 현재 모델의 성능이 이 값보다 나은 경우에만 이미 저장된 모델 가중치를 덮어쓴다.

    - `**kwargs`: 이전 버전과의 호환성을 위한 추가 인수이다. 가능한 키는 `.`이다.

**예제:**

```python
model.compile(loss=..., optimizer=...,
              metrics=['accuracy'])

EPOCHS = 10
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# 모델 가중치는 지금까지 가장 잘 보이는 경우 매 에포크가 끝날 때마다 저장된다.
model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

# 모델 가중치(가장 좋은 것으로 간주됨)가 모델에 적재된다.
model.load_weights(checkpoint_filepath)
```